In [105]:
import pandas as pd
import re
import json
pandas.options.display.precision("display.precision", 8)

NameError: name 'pandas' is not defined

In [106]:
def get_tensor_info(line):
    match = re.search("\w*:\s*\{'numel':\s*(\d+\.?\d?).*", line)
    if match:
        # print(match.groups())
        numel=int(match.group(1))
        return numel,
       
def get_median(line):
    match = re.search("Median:\s*(\d+\.?\d?)\w*", line)
    if match:
        # print(match.groups())
        median = float(match.group(1))
        return median

def get_IQR(line):
    # match = re.search("IQR:\s*(\d+\.?\d?)\s*\w*\s*\((\d+\.?\d?)\s*\w*\s*(\d+\.?\d?)\).*", line)
    match = re.search("IQR:\s*(\d+\.\d+)\s*\w*\s*\((\d+\.\d+)\s*\w*\s*(\d+\.\d+).*\)", line)
    if match:
        # print(match.groups())
        iqr = float(match.group(1))
        bottom = float(match.group(2))
        top = float(match.group(3))
        return iqr,


def read_fuzzer_log(path):

    # Using readlines()
    file1 = open(path, 'r')
    Lines = file1.readlines()

    count = 0
    # Strips the newline character
    data = []
    row = []
    for line in Lines:
        count += 1
        line = line.strip()

        if line:
            if re.search("x:(.*)", line):
                row+=get_tensor_info(line)
            elif re.search("y:(.*)", line):
                row+=get_tensor_info(line)
            elif re.search("Median:(.*)", line):
                row.append(get_median(line))
            elif re.search("IQR:(.*)", line):
                # row+=get_IQR(line)
                pass
        else:
            data.append(row)
            row = []

    # df= pd.DataFrame(data, columns=["x numel","y numel","Median","IQR(interquartile range)","low","high"])
    df=pd.DataFrame(data, columns=["x numel","y numel","Median"])
    # df=pd.DataFrame(data)

    df=df.set_index(["x numel","y numel"])
    return df



# path = "/home/fpadmin/dockerx/pytorch/fix_warpsize_issue_pt1_scripts_gfx908_128/fuzzer.log"
gfx908_128 = read_fuzzer_log("/home/fpadmin/dockerx/pytorch/fix_warpsize_issue_pt1_scripts_gfx908_gfx1030_128/fuzzer.log")
gfx908_256 = read_fuzzer_log("/home/fpadmin/dockerx/pytorch/fix_warpsize_issue_pt1_scripts_gfx908_gfx1030_256/fuzzer.log")
gfx1030_128 = read_fuzzer_log("/home/fpadmin/dockerx/pytorch/fix_warpsize_issue_pt1_scripts_gfx1030_gfx908_128/fuzzer.log")
gfx1030_256 = read_fuzzer_log("/home/fpadmin/dockerx/pytorch/fix_warpsize_issue_pt1_scripts_gfx1030_gfx908_256/fuzzer.log")


In [107]:
median_df=pd.concat([gfx908_128, gfx908_256, gfx1030_128, gfx1030_256], axis=1).set_axis(["gfx908_128(us)", "gfx908_256(us)", "gfx1030_128(us)", "gfx1030_256(us)"],axis=1)
median_df

,,gfx908_128(us),gfx908_256(us),gfx1030_128(us),gfx1030_256(us)
x numel,y numel,,,,
4359168,30272,109.7,110.1,143.3,143.6
16384,1,5.1,5.1,4.8,4.9
7307264,7307264,89.1,104.2,72.5,73.7
11239424,11239424,1.0,1.0,1.6,1.6
479196,479196,72.7,72.5,59.2,59.2


In [109]:
median_df.to_excel("median_df.xlsx")

ModuleNotFoundError: No module named 'openpyxl'